# Segmenting and Clustering Neighborhoods in Toronto Canada

## Peer Graded Assignment Part 3 (GKS)

### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

(1) to add enough Markdown cells to explain what you decided to do and to report any observations you make.

(2) to generate maps to visualize your neighborhoods and how they cluster together.

In [62]:
# install geopy & folium
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



#### Add Latitude and Longitude to the Postal Code DF

In [63]:
!pip install beautifulsoup4

In [64]:
import json # library to handle JSON files


In [65]:
# import Libraries that we need

import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON filesfrom geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

#### Data Scrape Wiki page

In [66]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [67]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [68]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3[df3.Borough != 'Not assigned']
df5 = df4.groupby(['Postal code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True) 
df6 = df5.replace("Not assigned", "Queen's Park")
df6.head()

,Postal code,Borough,,,,Neighborhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,Regent Park / Harbourfront,
3,M6A,North York,,,,Lawrence Manor / Lawrence Heights,
4,M7A,Downtown Toronto,,,,Queen's Park / Ontario Provincial Government,


##### Using the .csv file that has the geo coords for each postal code

In [69]:
url = "http://cocl.us/Geospatial_data"
df7 = pd.read_csv(url)
df7.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Change the first column name to reflect the assignment name

In [70]:
df7.rename(columns={'Postal Code': 'Postal code'}, inplace=True)
df7.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Create new dataframe to merge the previous two

In [71]:
df8 = pd.merge(df6, df7, on='Postal code')
df8.head(12)

,Postal code,Borough,,,,Neighborhood,,Latitude,Longitude
0,M3A,North York,,,,Parkwoods,,43.753259,-79.329656
1,M4A,North York,,,,Victoria Village,,43.725882,-79.315572
2,M5A,Downtown Toronto,,,,Regent Park / Harbourfront,,43.654260,-79.360636
3,M6A,North York,,,,Lawrence Manor / Lawrence Heights,,43.718518,-79.464763
4,M7A,Downtown Toronto,,,,Queen's Park / Ontario Provincial Government,,43.662301,-79.389494
5,M9A,Etobicoke,,,,Islington Avenue,,43.667856,-79.532242
6,M1B,Scarborough,,,,Malvern / Rouge,,43.806686,-79.194353
7,M3B,North York,,,,Don Mills,,43.745906,-79.352188
8,M4B,East York,,,,Parkview Hill / Woodbine Gardens,,43.706397,-79.309937
9,M5B,Downtown Toronto,,,,"Garden District, Ryerson",,43.657162,-79.378937


### Show how many Boroughs & Neighbourhoods in the dataframe

In [72]:
print('The dataframe has {} Boroughs and {} Neighbourhoods.'.format(
        len(df8['Borough'].unique()),
        df8.shape[0]
    )
)

The dataframe has 10 Boroughs and 103 Neighbourhoods.


In [73]:
Toronto=df8[df8['Borough'].str.contains('Toronto')]
Toronto

,Postal code,Borough,,,,Neighborhood,,Latitude,Longitude
2,M5A,Downtown Toronto,,,,Regent Park / Harbourfront,,43.654260,-79.360636
4,M7A,Downtown Toronto,,,,Queen's Park / Ontario Provincial Government,,43.662301,-79.389494
9,M5B,Downtown Toronto,,,,"Garden District, Ryerson",,43.657162,-79.378937
15,M5C,Downtown Toronto,,,,St. James Town,,43.651494,-79.375418
19,M4E,East Toronto,,,,The Beaches,,43.676357,-79.293031
20,M5E,Downtown Toronto,,,,Berczy Park,,43.644771,-79.373306
24,M5G,Downtown Toronto,,,,Central Bay Street,,43.657952,-79.387383
25,M6G,Downtown Toronto,,,,Christie,,43.669542,-79.422564
30,M5H,Downtown Toronto,,,,Richmond / Adelaide / King,,43.650571,-79.384568
31,M6H,West Toronto,,,,Dufferin / Dovercourt Village,,43.669005,-79.442259


In [74]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada is 43.6534817, -79.3839347.


In [75]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], 
                                           Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

#### For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in West Toronto. So let's slice the original dataframe and create a new dataframe of the West data.

In [76]:
WestToronto_data = df8[df8['Borough'] == 'West Toronto'].reset_index(drop=True)
WestToronto_data.head()

,Postal code,Borough,,,,Neighborhood,,Latitude,Longitude
0,M6H,West Toronto,,,,Dufferin / Dovercourt Village,,43.669005,-79.442259
1,M6J,West Toronto,,,,Little Portugal / Trinity,,43.647927,-79.419750
2,M6K,West Toronto,,,,Brockton / Parkdale Village / Exhibition Place,,43.636847,-79.428191
3,M6P,West Toronto,,,,High Park / The Junction South,,43.661608,-79.464763
4,M6R,West Toronto,,,,Parkdale / Roncesvalles,,43.648960,-79.456325


###### Let's get the geographical coordinates of West Toronto

In [77]:
address = 'West Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Toronto is 43.6534817, -79.3839347.


###### Let's visualize West Toronto neighborhoods.

In [78]:
# create map of West Toronto using latitude and longitude values
map_WestToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(WestToronto_data['Latitude'], WestToronto_data['Longitude'], WestToronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_WestToronto)  
    
map_WestToronto

####  Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [79]:
CLIENT_ID = 'XIKAQI5IHAOMJF0K5DIXZLA5GSD5RSWJP4WUTNYUPMSY5HWV' # your Foursquare ID
CLIENT_SECRET = '2AE3TUMNFMCWDR5JLG4FTY5X4AFFVZPKCD15LOYHFYK3PC3C' # your Foursquare Secret
VERSION = '20202204' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XIKAQI5IHAOMJF0K5DIXZLA5GSD5RSWJP4WUTNYUPMSY5HWV
CLIENT_SECRET:2AE3TUMNFMCWDR5JLG4FTY5X4AFFVZPKCD15LOYHFYK3PC3C


#### Let's explore the first neighborhood in our dataframe

In [80]:
WestToronto_data.loc[0, 'Neighborhood']

'Dufferin / Dovercourt Village'

###### Get the neighborhood's latitude and longitude values.

In [81]:
neighborhood_latitude = WestToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = WestToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = WestToronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dufferin / Dovercourt Village are 43.66900510000001, -79.4422593.


###### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters

In [82]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=XIKAQI5IHAOMJF0K5DIXZLA5GSD5RSWJP4WUTNYUPMSY5HWV&client_secret=2AE3TUMNFMCWDR5JLG4FTY5X4AFFVZPKCD15LOYHFYK3PC3C&v=20202204&ll=43.66900510000001,-79.4422593&radius=500&limit=100'

###### Send the GET request and examine the resutls

In [83]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea0a8436001fe001c69af98'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davenport',
  'headerFullLocation': 'Davenport, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.67350510450001,
    'lng': -79.43604977526607},
   'sw': {'lat': 43.664505095500004, 'lng': -79.44846882473394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5753753b498eeb535c53aed5',
       'name': 'The Greater Good Bar',
       'location': {'address': '229 Geary St',
        'crossStreet': 'at Dufferin St',
        'lat': 43.669409,
        'lng': -79.439267,
        'labeledLatLngs': [{'label': 'disp

###### Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [84]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [86]:
# install geopy & folium
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [87]:
from pandas.io.json import json_normalize

In [88]:


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Planet Fitness,Gym / Fitness Center,43.667588,-79.442574
3,Blood Brothers Brewing,Brewery,43.669944,-79.436533
4,FreshCo,Grocery Store,43.667918,-79.440754


###### And how many venues were returned by Foursquare?

In [89]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


## Let's Explore Neighborhoods in West Toronto, Canada

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###### Now write the code to run the above function on each neighborhood and create a new dataframe called WestToronto_venues

In [91]:
WestToronto_venues = getNearbyVenues(names=WestToronto_data['Neighborhood'],
                                   latitudes=WestToronto_data['Latitude'],
                                   longitudes=WestToronto_data['Longitude']
                                  )



Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / The Junction South
Parkdale / Roncesvalles
Runnymede / Swansea


###### Let's check the size of the resulting dataframe

In [92]:
print(WestToronto_venues.shape)
WestToronto_venues.head()

(153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dufferin / Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dufferin / Dovercourt Village,43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,Dufferin / Dovercourt Village,43.669005,-79.442259,Planet Fitness,43.667588,-79.442574,Gym / Fitness Center
3,Dufferin / Dovercourt Village,43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
4,Dufferin / Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store


Let's check how many venues were returned for each neighborhood

In [94]:
WestToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Dufferin / Dovercourt Village,16,16,16,16,16,16
High Park / The Junction South,24,24,24,24,24,24
Little Portugal / Trinity,41,41,41,41,41,41
Parkdale / Roncesvalles,13,13,13,13,13,13
Runnymede / Swansea,37,37,37,37,37,37


In [96]:
print('There are {} uniques categories.'.format(len(WestToronto_venues['Venue Category'].unique())))

There are 80 uniques categories.


## Let's Analyze each Neighborhood.

In [97]:
# one hot encoding
WestToronto_onehot = pd.get_dummies(WestToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
WestToronto_onehot['Neighborhood'] = WestToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [WestToronto_onehot.columns[-1]] + list(WestToronto_onehot.columns[:-1])
WestToronto_onehot = WestToronto_onehot[fixed_columns]

WestToronto_onehot.head()

,Neighborhood,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pub,Record Shop,Restaurant,Sandwich Place,Speakeasy,Stadium,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Dufferin / Dovercourt Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
# Let's check the size of the resulting dataframe
print(WestToronto_venues.shape)
WestToronto_venues.head()

(153, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dufferin / Dovercourt Village,43.669005,-79.442259,The Greater Good Bar,43.669409,-79.439267,Bar
1,Dufferin / Dovercourt Village,43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
2,Dufferin / Dovercourt Village,43.669005,-79.442259,Planet Fitness,43.667588,-79.442574,Gym / Fitness Center
3,Dufferin / Dovercourt Village,43.669005,-79.442259,Blood Brothers Brewing,43.669944,-79.436533,Brewery
4,Dufferin / Dovercourt Village,43.669005,-79.442259,FreshCo,43.667918,-79.440754,Grocery Store


In [99]:
# Let's check how many venues were returned for each neighborhood
WestToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Dufferin / Dovercourt Village,16,16,16,16,16,16
High Park / The Junction South,24,24,24,24,24,24
Little Portugal / Trinity,41,41,41,41,41,41
Parkdale / Roncesvalles,13,13,13,13,13,13
Runnymede / Swansea,37,37,37,37,37,37


In [100]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(WestToronto_venues['Venue Category'].unique())))

There are 80 uniques categories.


## Analyze Each Neighborhood

In [110]:
# one hot encoding
WestToronto_onehot = pd.get_dummies(WestToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
WestToronto_onehot['Neighborhood'] = WestToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [WestToronto_onehot.columns[-1]] + list(WestToronto_onehot.columns[:-1])
WestToronto_onehot = WestToronto_onehot[fixed_columns]

WestToronto_onehot.head()

,Neighborhood,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pub,Record Shop,Restaurant,Sandwich Place,Speakeasy,Stadium,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Dufferin / Dovercourt Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Dufferin / Dovercourt Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [111]:
WestToronto_onehot.shape

(153, 81)

###### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [112]:
WestToronto_grouped = WestToronto_onehot.groupby('Neighborhood').mean().reset_index()
WestToronto_grouped

,Neighborhood,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Burrito Place,Café,Cajun / Creole Restaurant,Climbing Gym,Cocktail Bar,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Venue,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pub,Record Shop,Restaurant,Sandwich Place,Speakeasy,Stadium,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Brockton / Parkdale Village / Exhibition Place,0.00000,0.000000,0.00000,0.045455,0.000000,0.045455,0.00000,0.000000,0.00000,0.090909,0.00000,0.045455,0.136364,0.000000,0.045455,0.00000,0.090909,0.045455,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.0000,0.000000,0.000000,0.000000,0.00000,0.045455,0.045455,0.0000,0.000000,0.00000,0.000000,0.045455,0.045455,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0000,0.00000,0.000000,0.000000,0.00000,0.045455,0.000000,0.045455,0.045455,0.000,0.000000,0.000000,0.00000,0.045455,0.000000,0.000000,0.045455,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0000,0.000000
1,Dufferin / Dovercourt Village,0.00000,0.000000,0.00000,0.125000,0.062500,0.062500,0.00000,0.000000,0.00000,0.000000,0.06250,0.000000,0.062500,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.00000,0.062500,0.000000,0.0625,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0625,0.00000,0.000000,0.062500,0.00000,0.000000,0.062500,0.000000,0.000000,0.125,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0625,0.000000
2,High Park / The Junction South,0.00000,0.041667,0.00000,0.041667,0.000000,0.041667,0.00000,0.041667,0.00000,0.000000,0.00000,0.000000,0.083333,0.041667,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.041667,0.041667,0.0000,0.041667,0.000000,0.000000,0.00000,0.083333,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.041667,0.00000,0.000000,0.00000,0.000000,0.041667,0.00000,0.083333,0.0000,0.00000,0.000000,0.041667,0.00000,0.000000,0.041667,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000000,0.000000,0.041667,0.000000,0.0000,0.000000,0.000000,0.000000,0.083333,0.00000,0.000000,0.00000,0.00000,0.0000,0.000000
3,Little Portugal / Trinity,0.02439,0.000000,0.04878,0.000000,0.000000,0.097561,0.02439,0.000000,0.02439,0.000000,0.02439,0.000000,0.048780,0.000000,0.000000,0.02439,0.024390,0.000000,0.024390,0.02439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.0000,0.000000,0.024390,0.000000,0.02439,0.000000,0.000000,0.0000,0.000000,0.02439,0.000000,0.000000,0.024390,0.02439,0.024390,0.02439,0.000000,0.000000,0.04878,0.000000,0.0000,0.02439,0.000000,0.000000,0.02439,0.000000,0.024390,0.000000,0.000000,0.000,0.024390,0.000000,0.02439,0.073171,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.02439,0

In [113]:
# Let's confirm new size
WestToronto_grouped.shape

(6, 81)

#### Let's print each neighborhood along with the top 5 most common venues

In [116]:
num_top_venues = 5

for hood in WestToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = WestToronto_grouped[WestToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton / Parkdale Village / Exhibition Place----
                    venue  freq
0                    Café  0.14
1             Coffee Shop  0.09
2          Breakfast Spot  0.09
3              Restaurant  0.05
4  Furniture / Home Store  0.05


----Dufferin / Dovercourt Village----
           venue  freq
0       Pharmacy  0.12
1         Bakery  0.12
2  Grocery Store  0.06
3        Brewery  0.06
4    Music Venue  0.06


----High Park / The Junction South----
                 venue  freq
0      Thai Restaurant  0.08
1        Grocery Store  0.08
2   Mexican Restaurant  0.08
3                 Café  0.08
4  Arts & Crafts Store  0.04


----Little Portugal / Trinity----
                   venue  freq
0                    Bar  0.10
1             Restaurant  0.07
2            Men's Store  0.05
3       Asian Restaurant  0.05
4  Vietnamese Restaurant  0.05


----Parkdale / Roncesvalles----
                         venue  freq
0                    Gift Shop  0.15
1                      Dog Run

#### Let's put that into a *pandas* dataframe


In [117]:
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [119]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = WestToronto_grouped['Neighborhood']

for ind in np.arange(WestToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(WestToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Italian Restaurant
1,Dufferin / Dovercourt Village,Bakery,Pharmacy,Gym / Fitness Center,Supermarket,Wine Shop,Gas Station,Middle Eastern Restaurant,Music Venue,Café,Brewery
2,High Park / The Junction South,Grocery Store,Thai Restaurant,Café,Mexican Restaurant,Diner,Speakeasy,Fast Food Restaurant,Liquor Store,Flea Market,Italian Restaurant
3,Little Portugal / Trinity,Bar,Restaurant,Men's Store,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Café,French Restaurant,Korean Restaurant,Juice Bar
4,Parkdale / Roncesvalles,Gift Shop,Cuban Restaurant,Breakfast Spot,Movie Theater,Eastern European Restaurant,Coffee Shop,Restaurant,Dog Run,Dessert Shop,Bookstore


##  Now, Let's Cluster Neighborhoods

In [120]:
# set number of clusters
kclusters = 5

WestToronto_grouped_clustering = WestToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(WestToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 1, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [125]:
# add clustering labels
WestToronto_merged = WestToronto_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
WestToronto_merged = WestToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
WestToronto_merged.head() # check the last columns!

,Postal code,Borough,,,,Neighborhood,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6H,West Toronto,,,,Dufferin / Dovercourt Village,,43.669005,-79.442259,2,Bakery,Pharmacy,Gym / Fitness Center,Supermarket,Wine Shop,Gas Station,Middle Eastern Restaurant,Music Venue,Café,Brewery
1,M6J,West Toronto,,,,Little Portugal / Trinity,,43.647927,-79.419750,1,Bar,Restaurant,Men's Store,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Café,French Restaurant,Korean Restaurant,Juice Bar
2,M6K,West Toronto,,,,Brockton / Parkdale Village / Exhibition Place,,43.636847,-79.428191,3,Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Italian Restaurant
3,M6P,West Toronto,,,,High Park / The Junction South,,43.661608,-79.464763,4,Grocery Store,Thai Restaurant,Café,Mexican Restaurant,Diner,Speakeasy,Fast Food Restaurant,Liquor Store,Flea Market,Italian Restaurant
4,M6R,West Toronto,,,,Parkdale / Roncesvalles,,43.648960,-79.456325,0,Gift Shop,Cuban Restaurant,Breakfast Spot,Movie Theater,Eastern European Restaurant,Coffee Shop,Restaurant,Dog Run,Dessert Shop,Bookstore


###### Finally, let's visualize the resulting clusters

In [127]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(WestToronto_merged['Latitude'], WestToronto_merged['Longitude'], WestToronto_merged['Neighborhood'], WestToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Now, Let's Examine the West Toronto Clusters

In [128]:
WestToronto_merged.loc[WestToronto_merged['Cluster Labels'] == 0, WestToronto_merged.columns[[1] + list(range(5, WestToronto_merged.shape[1]))]]

,Borough,Neighborhood,,,,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,West Toronto,Parkdale / Roncesvalles,,,,,43.64896,-79.456325,0,Gift Shop,Cuban Restaurant,Breakfast Spot,Movie Theater,Eastern European Restaurant,Coffee Shop,Restaurant,Dog Run,Dessert Shop,Bookstore


In [129]:
WestToronto_merged.loc[WestToronto_merged['Cluster Labels'] == 1, WestToronto_merged.columns[[1] + list(range(5, WestToronto_merged.shape[1]))]]

,Borough,Neighborhood,,,,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,West Toronto,Little Portugal / Trinity,,,,,43.647927,-79.41975,1,Bar,Restaurant,Men's Store,Asian Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Café,French Restaurant,Korean Restaurant,Juice Bar
5,West Toronto,Runnymede / Swansea,,,,,43.651571,-79.48445,1,Café,Pizza Place,Pub,Italian Restaurant,Coffee Shop,Sushi Restaurant,Juice Bar,Diner,Dessert Shop,Falafel Restaurant


In [130]:
WestToronto_merged.loc[WestToronto_merged['Cluster Labels'] == 2, WestToronto_merged.columns[[1] + list(range(5, WestToronto_merged.shape[1]))]]

,Borough,Neighborhood,,,,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,Dufferin / Dovercourt Village,,,,,43.669005,-79.442259,2,Bakery,Pharmacy,Gym / Fitness Center,Supermarket,Wine Shop,Gas Station,Middle Eastern Restaurant,Music Venue,Café,Brewery


In [131]:
WestToronto_merged.loc[WestToronto_merged['Cluster Labels'] == 3, WestToronto_merged.columns[[1] + list(range(5, WestToronto_merged.shape[1]))]]

,Borough,Neighborhood,,,,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,West Toronto,Brockton / Parkdale Village / Exhibition Place,,,,,43.636847,-79.428191,3,Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Italian Restaurant


In [132]:
WestToronto_merged.loc[WestToronto_merged['Cluster Labels'] == 4, WestToronto_merged.columns[[1] + list(range(5, WestToronto_merged.shape[1]))]]

,Borough,Neighborhood,,,,,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,West Toronto,High Park / The Junction South,,,,,43.661608,-79.464763,4,Grocery Store,Thai Restaurant,Café,Mexican Restaurant,Diner,Speakeasy,Fast Food Restaurant,Liquor Store,Flea Market,Italian Restaurant
